In [1]:
import sys
sys.path.insert(0, "..")

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# Data Processing
In this notebook, we will be processing the data in the ACM Dataset.

The first step is to understand the information we want to extract from our dataset.

In [5]:
PATH="../data/ACM Data/test/"

In [6]:
import glob
files = glob.glob(f'{PATH}/*.xml')
files[:5]

['../data/ACM Data/test/PROC--1969-800165.xml',
 '../data/ACM Data/test/PROC--1967-2402536.xml',
 '../data/ACM Data/test/PROC--1970-800028.xml']

In [10]:
from urop.xml import ACMCorpus
from gensim import corpora

texts = ACMCorpus(files)
dct = corpora.Dictionary(texts)

2018-03-08 04:40:03,391 : INFO : Parsing ../data/ACM Data/test/PROC--1969-800165.xml
2018-03-08 04:40:03,422 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-03-08 04:40:07,412 : INFO : Parsing ../data/ACM Data/test/PROC--1967-2402536.xml
2018-03-08 04:40:10,683 : INFO : Parsing ../data/ACM Data/test/PROC--1970-800028.xml
2018-03-08 04:40:12,614 : INFO : built Dictionary(6411 unique tokens: ['abstract', 'approximatey', 'ascii', 'bell', 'bit']...) from 75 documents (total 56913 corpus positions)


In [11]:
corpus = [dct.doc2bow(text) for text in texts]

2018-03-08 04:40:28,244 : INFO : Parsing ../data/ACM Data/test/PROC--1969-800165.xml
2018-03-08 04:40:31,970 : INFO : Parsing ../data/ACM Data/test/PROC--1967-2402536.xml
2018-03-08 04:40:35,098 : INFO : Parsing ../data/ACM Data/test/PROC--1970-800028.xml


In [8]:
from gensim.models import LdaModel

In [9]:
model = LdaModel(corpus=corpus, id2word=dct, num_topics=30)

2018-03-08 04:38:45,784 : INFO : using symmetric alpha at 0.03333333333333333
2018-03-08 04:38:45,785 : INFO : using symmetric eta at 0.03333333333333333
2018-03-08 04:38:45,786 : INFO : using serial LDA version on this node
2018-03-08 04:38:45,821 : WARNING : input corpus stream has no len(); counting documents
2018-03-08 04:38:45,822 : INFO : Parsing ../data/ACM Data/test/PROC--1969-800165.xml
2018-03-08 04:38:49,579 : INFO : Parsing ../data/ACM Data/test/PROC--1967-2402536.xml
2018-03-08 04:38:52,633 : INFO : Parsing ../data/ACM Data/test/PROC--1970-800028.xml
2018-03-08 04:38:54,442 : INFO : running online (single-pass) LDA training, 30 topics, 1 passes over the supplied corpus of 75 documents, updating model once every 75 documents, evaluating perplexity every 75 documents, iterating 50x with a convergence threshold of 0.001000
2018-03-08 04:38:54,442 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accurac

ValueError: too many values to unpack (expected 2)